In [1]:
from Environment.custom_env import CustomEnvironment
import json
import torch
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from Models.ActorCritic import ActorCritic
from Models.SoftActorCritic import SoftActorCritic
from Models.DDQN import DoubleDQN

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
torch.cuda.is_available()

True

In [3]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

<function TextIOWrapper.close()>

In [4]:
env = CustomEnvironment(env_config=env_variables, render_mode="human")
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

In [5]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler(feature_range=(0, 128))

In [6]:
ac_config = open('Environment/ac_config.json')
ac_variables = json.load(ac_config)
ac_config.close

sac_config = open('Environment/sac_config.json')
sac_variables = json.load(sac_config)
sac_config.close

ddqn_config = open('Environment/ddqn_config.json')
ddqn_variables = json.load(ddqn_config)
ddqn_config.close

<function TextIOWrapper.close()>

In [7]:
ddqn_1 = DoubleDQN(ddqn_variables)
# ddqn_2 = DoubleDQN(ddqn_variables)
# ddqn_3 = DoubleDQN(ddqn_variables)
# ac = ActorCritic(ac_variables)
# sac = SoftActorCritic(sac_variables)

In [8]:
models = {0:ddqn_1, 1:ddqn_1, 2:ddqn_1}

In [9]:
def plot_grad_flow(named_parameters):
    '''Plots the gradients flowing through different layers in the net during training.
    Can be used for checking for possible gradient vanishing / exploding problems.
    
    Usage: Plug this function in Trainer class after loss.backwards() as 
    "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
    ave_grads = []
    max_grads= []
    layers = []
    for i, p in enumerate(named_parameters):
        
        if(p.requires_grad):
            layers.append(i)
            ave_grads.append(p.grad.cpu().abs().mean())
            max_grads.append(p.grad.cpu().abs().max())

    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    plt.legend([Line2D([0], [0], color="c", lw=4),
                Line2D([0], [0], color="b", lw=4),
                Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])
    plt.show()

In [10]:
episodes = 400

for episode in tqdm(range(episodes)): #episodes loop"

    curr_obs,infos = env.reset() #reset environment after each episode\n",
    # print(curr_obs[0][1])

    losses = {}
    while(env.agents): #until there are any surviving agents 

        actions = {}
        log_probs = {}

        for a in env.agents:
            # print(curr_obs[a])
            act, log_prob = models[a].select_action(state = torch.tensor(curr_obs[a], dtype = torch.float32))
            actions[a] = act
            log_probs[a] = log_prob

        # for i in range(env.n_agents): #objects contains all of the models, get the corresponding actions from each policy\n",
            
        #     if i in env.agents:
        #         act, log_prob = env.agents_objects[i].brain1.select_action(state = torch.tensor(curr_obs[i], dtype = torch.float32))
        #         actions[i] = act
        #         log_probs[i] = log_prob
        #     else:
        #         actions[i] = env.num_actions
        #         log_probs[i] = 69
            
        # print(actions)

        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        if env.timestep % 50 == 0:
            # plot_grad_flow(models[0].policy_net.parameters())
            print(rewards)
        

        for id, obs in next_obs.items():
            models[id].push_to_buffer(torch.tensor(curr_obs[id], dtype = torch.float32),actions[id],rewards[id],torch.tensor(obs, dtype = torch.float32),log_probs[id],terminations[id])
            loss = models[id].update_weights()

            if(terminations[id] == True):
                # loss = models[id].update_weights()
                losses[id] = loss
        
        # for i in range(env.n_agents):

        #     if i in env.agents:
        #         env.agents_objects[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
        #         env.agents_objects[i].brain1.update_weights() 
            
        #     if i in env.justdie:
        #         env.justdie[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
        #         loss = env.justdie[i].brain1.update_weights() 
        #         del env.justdie[i]
        #         losses[i] = loss
        
        curr_obs = next_obs
    
    print(f"Episode {episode} end")
    print(len(ddqn_1.replay_buffer))
    print(f"Losses {losses}")
    print()
    env.close()

  0%|          | 0/400 [00:00<?, ?it/s]

{0: 0.57361960959597, 1: 0.41840087908508217, 2: 0.5500019841413828}
agent1 died at time 99
agent2 died at time 99
{0: 0.8336116135629557, 1: 0.6759987512175292, 2: 0.49534470079100246}
agent0 died at time 120
Episode 0 end
321
Losses {1: tensor(92.2216), 2: tensor(23.9393), 0: tensor(125.9719)}

{0: 0.7916823780303901, 1: 0.8077581641352923, 2: 0.6154808045809317}
agent0 died at time 99
{0: 0.6157288555237312, 1: -3, 2: 0.7563145934866299}
agent1 died at time 141
{2: 0.783925396360568}
{2: -3}
{2: -3}
agent2 died at time 267
Episode 1 end
831
Losses {0: tensor(19.2773), 1: tensor(14.3475), 2: tensor(30.0516)}

{0: 0.8399272996240852, 1: 0.6087921146182202, 2: 0.6016274766090621}
{0: 0.42334157657691474, 1: 0.6543582860635688, 2: 0.6488736037190399}
agent0 died at time 120
agent1 died at time 120
agent2 died at time 120
Episode 2 end
1194
Losses {0: tensor(218.6519), 1: tensor(149.0945), 2: tensor(24.4748)}

{0: 0.42935673448467426, 1: 0.6106648398075893, 2: 0.7181508266215624}
agent2 

KeyboardInterrupt: 

In [ ]:
models[0].steps_done